## Imports

In [ ]:

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

import scikitplot as skplt

from matplotlib.lines import Line2D
import seaborn as sns
import kds
import shap

from sklearn.base import BaseEstimator, TransformerMixin
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as imbPipeline
from skopt import BayesSearchCV


from feature_engine.imputation import AddMissingIndicator, CategoricalImputer
from feature_engine.encoding import CountFrequencyEncoder,OrdinalEncoder
from sklearn.preprocessing import FunctionTransformer,LabelEncoder

from sklearn.metrics import accuracy_score

from scikitplot.metrics import  plot_confusion_matrix

from xgboost import XGBClassifier


In [ ]:
df = pd.read_csv('data.csv')


## Pipelines
Se crean variables para unificar y entrenar el modelo a ver si es cancer o no.

In [ ]:
df["is_cancer"] = df["diagnostic"].apply(lambda x: 1 if x in ["BCC","MEL","SCC"] else 0)

In [ ]:
df.columns

In [ ]:
# la persona estuvo aca antes? Tuvo en esta parte del cuerpo antes? Que tuvo antes? -> me parece que va a overfitear con eso


x = df.drop(columns=["is_cancer", "diagnostic"]) 
y1 = df["is_cancer"]
y2 = df["diagnostic"]

class HadBeforeTransformer(BaseEstimator, TransformerMixin):
    def __init__(self) -> None:
        super().__init__()

    def fit(self, x:pd.DataFrame, y):
        self.patient_counts = x["patient_id"].value_counts()
        #x["had_here_before"] = x["patient_id"].apply(lambda x: self.patient_counts[x] -1  )
        return self
    def transform(self, x:pd.DataFrame):
        new_counts = x["patient_id"].value_counts()
        if new_counts.equals( self.patient_counts):
            new_counts = {}

        x["had_here_before"] = x["patient_id"].apply(lambda x: self.patient_counts.get(x,0)+ new_counts.get(x,0) -1   )
        return x
    
class HadThisPartBeforeTransformer(BaseEstimator, TransformerMixin):
    def __init__(self) -> None:
        super().__init__()

    def fit(self, x:pd.DataFrame, _):
        self.patient_region_counts = x.groupby(["patient_id", "region"]).size().unstack(fill_value=0).T.to_dict('dict')
        return self
    def transform(self, x:pd.DataFrame):
        new_counts = x.groupby(["patient_id", "region"]).size().unstack(fill_value=0).T.to_dict('dict')
        if new_counts == self.patient_region_counts:
            new_counts = {}
        x["had_this_part_before"] = x.apply(lambda row: 1 if  new_counts.get(row["patient_id"],{}).get(row["region"],0) + self.patient_region_counts.get(row["patient_id"],{}).get(row["region"],0)>=2 else 0, axis=1)
        return x
    


class NormalDistributionImputer(BaseEstimator, TransformerMixin):
    def __init__(self, variables=None):
        if not isinstance(variables, list):
            self.variables = [variables]
        else:
            self.variables = variables

    def fit(self, X, y=None):
        # we will store the mean and std of the variables for prediction time
        self.param_dict_ = {}
        for var in self.variables:
            self.param_dict_[var] = {'mean': X[var].mean(), 'std': X[var].std()}
        return self

    def transform(self, X):
        #TODO: Agregar seed para random
        X = X.copy()
        for feature in self.variables:
            mu, std = self.param_dict_[feature]['mean'], self.param_dict_[feature]['std']
            X[feature] = X[feature].apply(lambda x: np.random.normal(loc=mu, scale=std) if pd.isnull(x) else x)
        return X

class ClassifierTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, classifier_class, new_column_name,
                  target_column=False, eta=0.3,max_depth=6 ,
                   scale_pos_weight=1,min_child_weight=1,gamma =0,
                   subsample=1,colsample_bytree=1,n_estimators=100):
        self.classifier_class = classifier_class
        self.eta = eta
        self.max_depth = max_depth
        self.scale_pos_weight = scale_pos_weight
        self.min_child_weight = min_child_weight
        self.subsample = subsample
        self.colsample_bytree = colsample_bytree
        self.n_estimators = n_estimators
        self.gamma = gamma
        self.classifier_kwargs = {"eta":eta, "max_depth":max_depth,"scale_pos_weight":scale_pos_weight}
        self.new_column_name = new_column_name
        self.target_column = target_column

        self.label_encoder = LabelEncoder()

    def fit(self, x, y):
        self.classifier = self.classifier_class(eta=self.eta, max_depth=self.max_depth,
        scale_pos_weight=self.scale_pos_weight,min_child_weight = self.min_child_weight,
        gamma = self.gamma,subsample=self.subsample,colsample_bytree=self.colsample_bytree,n_estimators=self.n_estimators )

        if self.target_column:
            y = y.apply(lambda x: 1 if x in ["BCC","MEL","SCC"] else 0)
        else:
            y = self.label_encoder.fit_transform(y)

        # Ensure x and y have the same length
        if len(x) != len(y):
            raise ValueError(f'Length of x ({len(x)}) does not match length of y ({len(y)})')

        self.classifier.fit(x, y)
        return self

    def transform(self, x):
        if not hasattr(self, 'classifier'):
            raise RuntimeError('You must call fit before calling transform')

        x_copy = x.copy()
        x_copy[self.new_column_name] = self.classifier.predict(x)
        return x_copy

    def predict(self, x):
        return self.label_encoder.inverse_transform(self.classifier.predict(x))
    
    



pipe = imbPipeline([
    ( "had_here_before", HadBeforeTransformer()),
    ( "had_this_part_before", HadThisPartBeforeTransformer()),
    ("dropear", FunctionTransformer(lambda x: x.drop(["img_id","lesion_id", "patient_id"], axis=1,))),
    ("frequency", CountFrequencyEncoder(encoding_method='frequency',variables=["region"])),
    ("missingIndicator",  AddMissingIndicator(variables=["smoke","drink","background_father",
                                        "background_mother","pesticide","gender","skin_cancer_history",
                                            "cancer_history", "has_piped_water","has_sewage_system",
                                            "itch","grew","hurt","changed","bleed","elevation","fitspatrick","diameter_1","diameter_2","age"])),
    ("fillNaNs", CategoricalImputer(imputation_method="missing", variables=["smoke","drink","background_father",
                                        "background_mother","pesticide","gender","skin_cancer_history",
                                            "cancer_history", "has_piped_water","has_sewage_system",
                                            "itch","grew","hurt","changed","bleed","elevation"])),
    ("ordinal", OrdinalEncoder(encoding_method="arbitrary", variables=["smoke","drink","background_father",
                                        "background_mother","pesticide","gender","skin_cancer_history",
                                            "cancer_history", "has_piped_water","has_sewage_system",
                                            "itch","grew","hurt","changed","bleed","elevation"])),
    ("imputar_numericas",NormalDistributionImputer(  variables= ["fitspatrick","diameter_1","diameter_2","age"])),
    ("clf",ClassifierTransformer(XGBClassifier, "is_cancer", target_column=True, scale_pos_weight=float(np.sum(y1 == 0))*100 / np.sum(y1 == 1))),
    ("smote",SMOTE()),
    ("clf2",ClassifierTransformer(XGBClassifier, "diagnostic",target_column=False))

])

In [ ]:
x1,x1_test,y1,y1_test = train_test_split(x,y2, test_size=0.2, random_state=42)



In [ ]:
pipe.set_params(**{'clf__eta': 0.01,
    'clf__max_depth': 10,
    'clf__min_child_weight': 2,
    'clf__gamma': 0,
    "clf__subsample": 1,
    "clf__colsample_bytree": 1,
    "clf__n_estimators": 361,


    'clf2__eta': 0.3,
    'clf2__max_depth': 13,
    'clf2__min_child_weight': 0,
    'clf2__gamma': 0,
    "clf2__subsample":1 ,
    "clf2__colsample_bytree": 1,
    "clf2__n_estimators": 1000,})
pipe.fit(x1,y1 )

## Modelos

In [ ]:
param_space = {
    'clf__eta': (0.01, 0.3, 'uniform'),
    'clf__max_depth': (1, 10, 'uniform'),
    'clf__min_child_weight': (0,2,"uniform"),
    'clf__gamma': (0,2,"uniform"),
    "clf__subsample": (0.5,1,"uniform"),
    "clf__colsample_bytree": (0.5,1,"uniform"),
    "clf__n_estimators": (10,1000,"uniform"),


    'clf2__eta': (0.001, 0.5, 'uniform'),
    'clf2__max_depth': (1, 18, 'uniform'),
    'clf2__min_child_weight': (0,2,"uniform"),
    'clf2__gamma': (0,10,"uniform"),
    "clf2__subsample": (0.5,1,"uniform"),
    "clf2__colsample_bytree": (0.5,1,"uniform"),
    "clf2__n_estimators": (10,5000,"uniform"),

}
# Create a BayesSearchCV object
opt = BayesSearchCV(
    pipe,
    param_space,
    n_iter=64,
    cv=5, 
    n_jobs=-1,  # use all available cores
    scoring='accuracy'
)

# Fit the BayesSearchCV object to the data
opt.fit(x1,y1)

# Print the best parameters and score
print("Best parameters found: ", opt.best_params_)
print("Best score found: ", opt.best_score_)


In [ ]:
y1_pred = pipe.predict(x1_test)

In [ ]:
#y1_pred = pipe.predict(x1_test)
encoder = LabelEncoder()
encoder.fit(y1_test)
accuracy_score(encoder.transform(y1_test), encoder.transform(y1_pred))

In [ ]:
#Confusion matrix
class_order = ['BCC','MEL','SCC','ACK','NEV','SEK']
plot_confusion_matrix(y1_test, y1_pred, normalize=False,labels=class_order)

## SHAP

In [ ]:
xgb  = pipe[-3].classifier

X_test:pd.DataFrame = pipe[:-2].transform(x1_test).drop(columns=["is_cancer"])
X_test["biopsed"] = X_test["biopsed"].apply(lambda x: 1 if x else 0)
X_train = pipe[:-2].transform(x1).drop(columns=["is_cancer"])
X_train["biopsed"] = X_train["biopsed"].apply(lambda x: 1 if x else 0)
explainer = shap.TreeExplainer(xgb,data=X_train)

In [ ]:


#print(df.corr()["diagnostic"].sort_values(ascending=False))


shap_values = explainer(X_train.values)

In [ ]:
shap.summary_plot(shap_values, X_train)

In [ ]:

def ABS_SHAP(df_shap,df):

    shap_v = pd.DataFrame(df_shap.copy())
    shap_v.columns = df.columns
    df_v = df.copy().reset_index().drop('index',axis=1)
    df_v = df_v.astype('float')
    # Determine the correlation in order to plot with different colors
    corr_list = list()
    feature_list = df.columns
    for i in feature_list:
        b = np.corrcoef(shap_v[i],df_v[i])[1][0]
        corr_list.append(b)
    corr_df = pd.concat([pd.Series(feature_list),pd.Series(corr_list)],axis=1).fillna(0)
    # Make a data frame. Column 1 is the feature, and Column 2 is the correlation coefficient
    corr_df.columns  = ['Variable','Corr']
    corr_df['Sign'] = np.where(corr_df['Corr']>0,'#ff0051','#0076f1')

    sns.set_style("whitegrid")
    
    shap_abs = np.abs(shap_v)
    k=pd.DataFrame(shap_abs.mean()).reset_index()
    k.columns = ['Variable','SHAP_abs']
    k2 = k.merge(corr_df,left_on = 'Variable',right_on='Variable',how='inner')
    k2 = k2.sort_values(by='SHAP_abs',ascending = False).head(10)
    k2.reset_index(inplace=True,drop=True)
    k2['SHAP_abs']=round(k2['SHAP_abs'],2)
    colorlist = k2['Sign']
    ax=sns.barplot(data=k2,y='Variable',x='SHAP_abs',palette=colorlist,)
    ax.set_xlabel("Mean SHAP Value")
    ax.legend([Line2D([0], [0], color='#ff0051', lw=4),Line2D([0], [0], color='#0076f1', lw=4)], ['Positiva', 'Negativa'],title='Correlación')


In [ ]:
ABS_SHAP(shap_values.values,X_train)

In [ ]:
y_pred_proba=xgb.predict_proba(X_test)[:,1]


# Get the index of the maximum probability
max_prob_index = y_pred_proba.argmax()
min_prob_index = y_pred_proba.argmin()

# Get the corresponding row in X_test
max_index_row = X_test.iloc[[max_prob_index]]
min_index_row = X_test.iloc[[min_prob_index]]

In [ ]:
# Compute SHAP values
shap_values_f = explainer(max_index_row)

# Create a waterfall plot
shap.plots.waterfall(shap_values_f[0])

In [ ]:
shap_values_f_min = explainer(min_index_row)

shap.plots.waterfall(shap_values_f_min[0])

In [ ]:
shap.initjs()

shap.plots.force(shap_values_f)

In [ ]:
shap.plots.force(shap_values_f_min)

## Metricas de clasificación

In [ ]:
y_test = encoder.transform(y1_test)


In [ ]:
kds.metrics.plot_cumulative_gain(y_test, y_pred_proba)

In [ ]:
kds.metrics.plot_lift(y_test, y_pred_proba)


In [ ]:
kds.metrics.plot_lift_decile_wise(y_test, y_pred_proba)

In [ ]:
kds.metrics.plot_ks_statistic(y_test, y_pred_proba)

In [ ]:
kds.metrics.report(y_test, y_pred_proba)

In [ ]:
import numpy as np

# Reshape y_pred_proba into a 2D array if it's currently a 1D array
if y_pred_proba.ndim == 1:
    y_pred_proba = np.column_stack([1 - y_pred_proba, y_pred_proba])

# Plot the precision-recall curve
skplt.metrics.plot_precision_recall(y_test, y_pred_proba, classes_to_plot=1, plot_micro=False)

In [120]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report, confusion_matrix
y_pred = y1_pred
y_test = y1_test
""" auc_macro=roc_auc_score(y_test, y_pred_proba, average="macro", multi_class="ovr")
auc_weighted=roc_auc_score(y_test, y_pred_proba, average="weighted", multi_class="ovr")
 """

' auc_macro=roc_auc_score(y_test, y_pred_proba, average="macro", multi_class="ovr")\nauc_weighted=roc_auc_score(y_test, y_pred_proba, average="weighted", multi_class="ovr")\n '

In [121]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         ACK       0.86      0.83      0.85       133
         BCC       0.83      0.91      0.87       177
         MEL       0.77      0.77      0.77        13
         NEV       0.77      0.72      0.74        50
         SCC       0.70      0.52      0.60        44
         SEK       0.61      0.63      0.62        43

    accuracy                           0.80       460
   macro avg       0.76      0.73      0.74       460
weighted avg       0.80      0.80      0.80       460



In [122]:
auc_macro=roc_auc_score(y_test, y_pred_proba, average="macro", multi_class="ovr")
auc_weighted=roc_auc_score(y_test, y_pred_proba, average="weighted", multi_class="ovr")

ValueError: Number of classes in y_true not equal to the number of columns in 'y_score'

In [126]:
#Método incluido en sklearn
from sklearn.metrics import cohen_kappa_score


accuracy=accuracy_score(y_test,y_pred)
print(f'Accuracy: {accuracy}')
print(f'Kappa Sklearn: {cohen_kappa_score(y_test, y_pred)}')

Accuracy: 0.8
Kappa Sklearn: 0.7256259643936152
